<a href="https://colab.research.google.com/github/qwerlarlgus/NLP/blob/main/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive 연동 설정

실습에 앞서, 모델 파일과 학습 코드가 저장되어 있는 구글 드라이브의 디렉토리와 Colab을 연동하겠습니다.

먼저, 좌측 상단 메뉴에서 런타임 -> 런타임 유형 변경 -> 하드웨어 가속기 -> GPU 선택 후 저장해주세요 :-)

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!nvidia-smi

Mon Feb 15 05:36:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    14W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/nlp/src-20210215T011122Z-001.zip

Archive:  /content/drive/MyDrive/nlp/src-20210215T011122Z-001.zip
  inflating: src/make_bert_model/run_multi_classifier.py  
  inflating: src/make_bert_model/run_squad.py  
  inflating: src/make_vocab/wordpiece.py  
  inflating: src/make_bert_model/run_classifier.py  
  inflating: src/make_preprocessed_data/tokenization.py  
  inflating: src/make_bert_model/modeling.py  
  inflating: src/make_bert_model/run_pretraining.py  
  inflating: src/make_preprocessed_data/__pycache__/tokenization.cpython-36.pyc  
  inflating: src/make_preprocessed_data/create_pretraining_data.py  
  inflating: src/make_bert_model/__pycache__/optimization.cpython-36.pyc  
  inflating: src/make_bert_model/__pycache__/modeling.cpython-36.pyc  
  inflating: src/make_bert_model/__pycache__/optimization.cpython-37.pyc  
  inflating: src/make_preprocessed_data/__pycache__/tokenization.cpython-37.pyc  
  inflating: src/make_bert_model/tokenization.py  
  inflating: src/make_bert_model/extract_features.py  
  inflating:

In [ ]:
!ls

drive  sample_data


In [ ]:
!ls drive/My\ Drive/bert_for_practics/t_academy

 딥러닝_기반_자연어_언어모델_BERT_ver_1.0.pdf   rsc   T_academy_bert.ipynb
'질의사항 답변.gdoc'			        src


In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==1.14

Uninstalling tensorflow-2.4.1:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.4.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
A

  Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 109.2MB 104kB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 3.2MB 55.6MB/s 
     |████████████████████████████████| 491kB 53.7MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully u

# BERT 학습을 위한 vocab을 만들기

학습에 사용될 vocab을 만들어보겠습니다.

아래 코드를 실행하시면, vocab을 만들 수 있습니다.

이번 예제에서는 결과를 빠르게 확인하기 위해 iter를 20으로 두었으나, 

wiki를 기준으로 1000 정도로 iter를 설정하면 대략 30,000 vocab정도 만들어집니다.

In [ ]:
!python /content/src/make_vocab/wordpiece.py \
--corpus=/content/rsc/training_data/wiki_20190620_small.txt \
--iter=20 \
--fname=/content/rsc/my_conf/my_vocab.txt

#BERT 학습을 위한 Preprocessed data 만들기

이제 vocab 이 준비되었으니, BERT 학습을 위한 corpus를 preprocessing 해보도록 하겠습니다.

In [ ]:
!python /content/src/make_preprocessed_data/create_pretraining_data.py \
--input_file=/content/rsc/training_data/wiki_20190620_small.txt \
--vocab_file=/content/rsc/conf/my_vocab.txt \
--do_lower_case=False \
--max_seq_length=512 \
--output_file=/content/rsc/my_preprocessed_training_data/wiki_20190620_small_512_tf.record

# BERT 학습

이제 만들어진 학습 데이터를 이용해서 실제로 BERT를 학습해보도록 하겠습니다.

이번 학습에선 앞서 만든 wiki_small이 아니라, 제가 미리 만들어 배포드린 전체 wiki 데이터를 이용해 학습해보겠습니다.

참고로, train_batch_size를 4보다 크게 할 경우, colab에서 제공하는 GPU로는 메모리가 부족해서 에러가 발생합니다.

In [ ]:
!python /content/src/make_bert_model/run_pretraining.py \
--input_file=/content/rsc/my_preprocessed_training_data/wiki_20190620_small_512_tf.record \
--output_dir=/content/rsc/my_pretrained_model \
--do_train=True \
--do_eval=True \
--bert_config_file=/content/rsc/conf/bert_config.json \
--train_batch_size=4 \
--max_seq_length=512 \
--max_predictions_per_seq=20 \
--num_train_steps=10 \
--learning_rate=1e-4 \
--save_checkpoints_steps=10 \
--do_lower_case=False

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

# 학습 된 BERT 모델로 KorQuAD 학습

이번엔 사전에 제공해드린 BERT 모델을 이용해 KorQuAD를 학습해보도록 하겠습니다.

max_seq_length와 num_train_epochs 등을 줄여서 학습 시간을 줄였지만, 

그래도 대략 30분 정도 소요됩니다 :-)

Default parameter로 학습을 하면 대략 4시간 정도 학습이 소요됩니다.

In [ ]:
!python /content/src/make_bert_model/run_squad.py \
--vocab_file=/content/rsc/conf/vocab.txt \
--bert_config_file=/content/rsc/conf/bert_config.json \
--init_checkpoint=/content/rsc/pretrained_model/model_output_512_model.ckpt-200000 \
--do_train=True \
--train_file=/content/rsc/KorQuAD/KorQuAD_v1.0_train.json \
--do_predict=True \
--predict_file=/content/rsc/KorQuAD/KorQuAD_v1.0_dev.json \
--train_batch_size=16 \
--learning_rate=2e-5 \
--num_train_epochs=1.0 \
--max_seq_length=128 \
--doc_stride=128 \
--output_dir=/content/rsc/KorQuAD_output \
--do_lower_case=False

이제 구글 드라이브에 predictions.json 이라는 이름으로 KorQuAD dev set에 대한 prediction 결과가 저장됐습니다.

# 학습 된 KorQuAD 평가

아래 코드를 실행해보시면, KorQuAD를 평가하실 수 있습니다.

결과가 실망스러우신가요? :-)

Defualt parameter로 KorQuAD 학습을 해보시면,

{"exact_match": 67.23242119847593, "f1": 86.38759335665746}

정도의 결과를 확인하실 수 있습니다 :-)

역시 높은 수치는 아니지만, wiki 데이터만을 이용해 20만 step만 학습한 BERT 모델로는

괜찮게 나온 결과라고 생각합니다 :-)

---



In [ ]:
!python /content/rsc/KorQuAD/evaluate-v1.0.py /content/rsc/KorQuAD/KorQuAD_v1.0_dev.json /content/rsc/KorQuAD_output/predictions.json

Traceback (most recent call last):
  File "/content/rsc/KorQuAD/evaluate-v1.0.py", line 119, in <module>
    with open(args.prediction_file) as prediction_file:
FileNotFoundError: [Errno 2] No such file or directory: '/content/rsc/KorQuAD_output/predictions.json'


# BERT 감정 데이터 분류 실습

이번에는 네이버 영화 리뷰 데이터와 BERT를 이용해 문장 분류를 실습해보겠습니다.

네이버 영화 리뷰 데이터는 아래의 주소에서 다운받을 수 있습니다.

https://github.com/e9t/nsmc

BERT에서는 run classification 코드를 통해 분류 학습을 수행할 수 있습니다.

약 20분 정도 소요됩니다 :-)

In [ ]:
!python /content/src/make_bert_model/run_classifier.py \
--task_name=nsmc \
--do_train=true \
--do_eval=true \
--data_dir=/content/rsc/nsmc \
--vocab_file=/content/rsc/conf/vocab.txt \
--bert_config_file=/content/rsc/conf/bert_config.json \
--init_checkpoint=/content/rsc/pretrained_model/model_output_512_model.ckpt-200000 \
--max_seq_length=128 \
--train_batch_size=32 \
--num_train_epochs=1.0 \
--learning_rate=3e-5 \
--do_lower_case=false \
--output_dir=/content/rsc/nsmc_output

정확도가 86%정도 나왔네요!

짧게 학습을 수행한 것 치고는 매우 높게 잘 나온 것 같습니다 :-)

# BERT 관계 추출 실습

이번 실습에서는 BERT를 이용해 entity가 가지는 관계를 추출해보는 실습을 해보도록 하겠습니다.

관계 추출 데이터는 Kaist가 공개한 데이터를 사용하였습니다.

해당 데이터는 다음의 사이트에서 다운로드 받으실 수 있습니다.

https://github.com/machinereading/kor-re-gold

본 테스트를 위해 학습 데이터 n건과 테스트 데이터 n건으로 나눠서 전처리를 수행해두었습니다.

기존 BERT와 달리, 이 경우엔 다음과 같이 문장이 입력됩니다.

Subject[SEP]Object[SEP]Sentence

In [ ]:
!python /content/src/make_bert_model/run_multi_classifier.py \
--task_name=kent \
--do_train=true \
--do_eval=true \
--data_dir=/content/rsc/relation \
--vocab_file=/content/rsc/conf/vocab.txt \
--bert_config_file=/content/rsc/conf/bert_config.json \
--init_checkpoint=/content/rsc/pretrained_model/model_output_512_model.ckpt-200000 \
--max_seq_length=128 \
--train_batch_size=32 \
--num_train_epochs=1.0 \
--learning_rate=2e-5 \
--do_lower_case=false \
--output_dir=/content/rsc/relation_output

학습 시간이 정말 오래걸리죠? :-)

발표 장표에 첨부된 85%의 결과는 거의 하루 가까이 돌렸던 것 같네요 :-)